# Script 01 — Foundations: LLM, MEMORY, THOUGHT

This script teaches the three core primitives of ThoughtFlow from scratch.
By the end you will understand how to:

- Connect to language models (local and cloud)
- Store and retrieve conversation state in MEMORY
- Create THOUGHTs that reason over memory with real LLM calls

**Every cell makes real API calls** — no mocks, no fakes.

Prerequisites:
  - Ollama running locally with a model pulled (`ollama pull llama3.2`)
  - A Groq API key (free at https://console.groq.com)
  - `pip install thoughtflow`

In [1]:
# --- Environment setup ---
# Load API keys from .env and make thoughtflow importable.

from _setup import load_env, print_heading, print_separator

env = load_env()
print("Loaded {} env vars".format(len(env)))

Loaded 3 env vars


In [2]:
from thoughtflow import LLM, MEMORY, THOUGHT
import os

---
## Part 1 — The LLM Primitive

`LLM` is a unified interface for language models.  You create one by
passing a **provider:model** string.  Supported providers include
`openai`, `anthropic`, `groq`, `gemini`, `openrouter`, and `ollama`.

We will start with **Ollama** (a free, local model) to prove that
ThoughtFlow works without any cloud dependency, then switch to
**Groq** for the rest of the demo.

In [3]:
print_heading("1.1  LLM with Ollama (local)")

# Ollama requires no API key — it runs on your machine.
# Make sure Ollama is running: `ollama serve`
ollama_llm = LLM("ollama:gemma3")

# The simplest possible call: pass a list of message dicts.
response = ollama_llm.call([
    {"role": "user", "content": "In one sentence, what is Python?"}
])

print("Provider :", ollama_llm.service)
print("Model    :", ollama_llm.model)
print("Response :", response[0])


══════════════════════════════════════════════════════════════════════
  1.1  LLM with Ollama (local)
══════════════════════════════════════════════════════════════════════

Provider : ollama
Model    : gemma3
Response : Python is a versatile, high-level programming language known for its readability and wide range of applications, from web development to data science.


In [4]:
print_heading("1.2  LLM with Groq (cloud)")

# Groq hosts open-source models with very fast inference.
groq_key = os.environ.get("GROQ_API_KEY", "")
llm = LLM("groq:llama-3.3-70b-versatile", key=groq_key)

response = llm.call([
    {"role": "user", "content": "In one sentence, what makes a good API?"}
])

print("Provider :", llm.service)
print("Model    :", llm.model)
print("Response :", response[0])


══════════════════════════════════════════════════════════════════════
  1.2  LLM with Groq (cloud)
══════════════════════════════════════════════════════════════════════

Provider : groq
Model    : llama-3.3-70b-versatile
Response : A good API is one that is well-documented, intuitive, and follows standard design principles, such as RESTful architecture, to provide a simple, secure, and scalable interface for interacting with a system or service.


In [5]:
print_heading("1.3  Message normalization")

# LLM.call() accepts several message formats.  It normalizes them
# internally so you do not have to worry about the exact shape.

# Format A: list of dicts (standard)
resp_a = llm.call([
    {"role": "system", "content": "You are terse.  Reply in ≤5 words."},
    {"role": "user", "content": "What color is the sky?"},
])
print("Dict messages :", resp_a[0])

# Format B: plain strings (auto-wrapped as role='user')
resp_b = llm.call(["What color is grass?  Reply in ≤5 words."])
print("String message:", resp_b[0])

# Format C: mixed — dicts and strings together
resp_c = llm.call([
    {"role": "system", "content": "You are terse.  Reply in ≤5 words."},
    "What color is the ocean?",
])
print("Mixed messages:", resp_c[0])


══════════════════════════════════════════════════════════════════════
  1.3  Message normalization
══════════════════════════════════════════════════════════════════════

Dict messages : Blue.
String message: Green.
Mixed messages: Blue.


---
## Part 2 — The MEMORY Primitive

`MEMORY` is an **event-sourced state container**.  Every change — a
message, a variable update, a log entry, a reflection — is stored as
an immutable event with a unique, sortable stamp.

Think of it as the agent's brain: it holds the full conversation,
all working variables, and a complete audit trail.

In [6]:
print_heading("2.1  Creating a MEMORY and adding messages")

memory = MEMORY()
print("Memory ID:", memory.id)

# Messages require a role and content.  The optional `channel` parameter
# tracks where the message came from (webapp, cli, telegram, etc.).
memory.add_msg("system", "You are a helpful research assistant.", channel="cli")
memory.add_msg("user", "Tell me about event sourcing.", channel="cli")
memory.add_msg("assistant", "Event sourcing stores every state change as an immutable event.", channel="cli")

print("\nMessages in memory:")
for msg in memory.get_msgs():
    print("  [{role}] {content}".format(**msg))


══════════════════════════════════════════════════════════════════════
  2.1  Creating a MEMORY and adding messages
══════════════════════════════════════════════════════════════════════

Memory ID: 51yDVOIsQpd6Slo0

Messages in memory:
  [system] You are a helpful research assistant.
  [user] Tell me about event sourcing.
  [assistant] Event sourcing stores every state change as an immutable event.


In [7]:
print_heading("2.2  Convenience accessors")

# Quick access to the most recent message by role.
print("Last user msg    :", memory.last_user_msg(content_only=True))
print("Last assistant msg:", memory.last_asst_msg(content_only=True))
print("Last system msg  :", memory.last_sys_msg(content_only=True))


══════════════════════════════════════════════════════════════════════
  2.2  Convenience accessors
══════════════════════════════════════════════════════════════════════

Last user msg    : Tell me about event sourcing.
Last assistant msg: Event sourcing stores every state change as an immutable event.
Last system msg  : You are a helpful research assistant.


In [8]:
print_heading("2.3  Variables — set, get, describe")

# Variables are the agent's working memory.  Each set_var call
# *appends* to the variable's history — it never overwrites.
memory.set_var("topic", "event sourcing", desc="The user's research topic")
memory.set_var("depth", "introductory", desc="How deep the user wants to go")
memory.set_var("sources_found", 0, desc="Number of sources found so far")

print("topic        :", memory.get_var("topic"))
print("depth        :", memory.get_var("depth"))
print("sources_found:", memory.get_var("sources_found"))

# Descriptions are tracked separately and are retrievable.
print("\nDescription of 'topic':", memory.get_var_desc("topic"))


══════════════════════════════════════════════════════════════════════
  2.3  Variables — set, get, describe
══════════════════════════════════════════════════════════════════════

topic        : event sourcing
depth        : introductory
sources_found: 0

Description of 'topic': The user's research topic


In [9]:
print_heading("2.4  Variable history")

# Every update to a variable is recorded.  Let us update sources_found
# a few times and then inspect the full history.
memory.set_var("sources_found", 3)
memory.set_var("sources_found", 7)
memory.set_var("sources_found", 12)

print("Current value:", memory.get_var("sources_found"))
print("\nFull history of 'sources_found':")
for stamp, value in memory.get_var_history("sources_found"):
    print("  {} → {}".format(stamp[:12], value))


══════════════════════════════════════════════════════════════════════
  2.4  Variable history
══════════════════════════════════════════════════════════════════════

Current value: 12

Full history of 'sources_found':
  51yDVr70jTYx → 0
  51yDW5khKuiY → 3
  51yDW5kjYAty → 7
  51yDW5kjUXJ2 → 12


In [10]:
print_heading("2.5  get_all_vars — snapshot of current state")

all_vars = memory.get_all_vars()
print("All current variables:")
for key, value in all_vars.items():
    print("  {} = {}".format(key, value))


══════════════════════════════════════════════════════════════════════
  2.5  get_all_vars — snapshot of current state
══════════════════════════════════════════════════════════════════════

All current variables:
  topic = event sourcing
  depth = introductory
  sources_found = 12


In [11]:
print_heading("2.6  Tombstone deletion")

# Deleting a variable does not erase it — it appends a tombstone marker.
# The full history is always preserved.
memory.del_var("depth")

print("After deletion:")
print("  get_var('depth')       :", memory.get_var("depth"))
print("  is_var_deleted('depth'):", memory.is_var_deleted("depth"))

# The history shows the deletion event.
print("\nFull history of 'depth':")
for stamp, value in memory.get_var_history("depth"):
    label = "<DELETED>" if value is not None and str(value).startswith("__") else str(value)
    print("  {} → {}".format(stamp[:12], label))

# Re-setting a deleted variable brings it back.
memory.set_var("depth", "intermediate")
print("\nAfter re-setting: get_var('depth') =", memory.get_var("depth"))


══════════════════════════════════════════════════════════════════════
  2.6  Tombstone deletion
══════════════════════════════════════════════════════════════════════

After deletion:
  get_var('depth')       : None
  is_var_deleted('depth'): True

Full history of 'depth':
  51yDVr6zu9Oi → introductory
  51yDWQ2s2c3J → <DELETED>

After re-setting: get_var('depth') = intermediate


In [12]:
print_heading("2.7  Multi-channel messages")

# Channels track where a message originated.  This matters when your
# agent talks to users across multiple platforms simultaneously.
memory.add_msg("user", "Hey from my phone!", channel="ios")
memory.add_msg("user", "Checking in on Telegram.", channel="telegram")

# Filter messages by channel.
print("Messages from 'ios':")
for msg in memory.get_msgs(channel="ios"):
    print("  [{role}] {content}".format(**msg))

print("\nMessages from 'telegram':")
for msg in memory.get_msgs(channel="telegram"):
    print("  [{role}] {content}".format(**msg))


══════════════════════════════════════════════════════════════════════
  2.7  Multi-channel messages
══════════════════════════════════════════════════════════════════════

Messages from 'ios':
  [user] Hey from my phone!

Messages from 'telegram':
  [user] Checking in on Telegram.


In [13]:
print_heading("2.8  Logs and reflections")

# Logs are internal bookkeeping.  Reflections are the agent's
# self-observations (useful for metacognitive architectures).
memory.add_log("Session started for user demo")
memory.add_log("Research topic identified: event sourcing")
memory.add_ref("The user seems interested in distributed systems concepts")
memory.add_ref("Should ask about their experience level next")

print("Logs:")
for log in memory.get_logs():
    print("  ", log["content"])

print("\nReflections:")
for ref in memory.get_refs():
    print("  ", ref["content"])


══════════════════════════════════════════════════════════════════════
  2.8  Logs and reflections
══════════════════════════════════════════════════════════════════════

Logs:
   Session started for user demo
   Research topic identified: event sourcing

Reflections:
   The user seems interested in distributed systems concepts
   Should ask about their experience level next


In [14]:
print_heading("2.9  Rendering memory for LLMs")

# render() with format='conversation' produces a compact text blob
# optimized for stuffing into an LLM prompt.
conversation_text = memory.render(format="conversation", max_total_length=1000)
print("Conversation render (first 500 chars):")
print(conversation_text[:500])

# prepare_context() gives you an OpenAI-compatible message list
# with smart truncation of older messages.
context = memory.prepare_context(recent_count=4, format="openai")
print("\n\nPrepared context ({} messages):".format(len(context)))
for msg in context:
    preview = msg["content"][:60]
    if len(msg["content"]) > 60:
        preview += "..."
    print("  [{}] {}".format(msg["role"], preview))


══════════════════════════════════════════════════════════════════════
  2.9  Rendering memory for LLMs
══════════════════════════════════════════════════════════════════════

Conversation render (first 500 chars):
User: Tell me about event sourcing.

Assistant: Event sourcing stores every state change as an immutable event.

User: Hey from my phone!

User: Checking in on Telegram.


Prepared context (4 messages):
  [user] Tell me about event sourcing.
  [assistant] Event sourcing stores every state change as an immutable eve...
  [user] Hey from my phone!
  [user] Checking in on Telegram.


In [15]:
print_heading("2.10  Persistence — save and load")

import tempfile, os

# JSON round-trip (portable, human-readable)
json_str = memory.to_json()
print("JSON export: {} chars".format(len(json_str)))

restored_from_json = MEMORY.from_json(json_str)
print("Restored from JSON — topic:", restored_from_json.get_var("topic"))

# Pickle round-trip (fast, binary)
tmp_path = os.path.join(tempfile.gettempdir(), "demo_memory.pkl")
memory.save(tmp_path)
print("\nSaved to:", tmp_path)

loaded_memory = MEMORY()
loaded_memory.load(tmp_path)
print("Loaded from pickle — topic:", loaded_memory.get_var("topic"))
print("Loaded from pickle — events:", len(loaded_memory.events))

# Clean up
os.remove(tmp_path)


══════════════════════════════════════════════════════════════════════
  2.10  Persistence — save and load
══════════════════════════════════════════════════════════════════════

JSON export: 9203 chars
Restored from JSON — topic: event sourcing

Saved to: /var/folders/vy/hdl322p54t5fg8ftxtl6hdl40000gn/T/demo_memory.pkl
Loaded from pickle — topic: event sourcing
Loaded from pickle — events: 17


In [16]:
print_heading("2.11  Snapshot and rehydration")

# snapshot() exports the full state.  from_events() rebuilds it.
# This is the mechanism that enables cloud sync.
snap = memory.snapshot()
print("Snapshot keys:", list(snap.keys()))
print("Event count  :", len(snap["events"]))

rehydrated = MEMORY.from_events(snap["events"].values(), memory.id, objects=snap["objects"])
print("Rehydrated OK — same topic?", rehydrated.get_var("topic") == memory.get_var("topic"))


══════════════════════════════════════════════════════════════════════
  2.11  Snapshot and rehydration
══════════════════════════════════════════════════════════════════════

Snapshot keys: ['id', 'events', 'objects']
Event count  : 17
Rehydrated OK — same topic? True


---
## Part 3 — The THOUGHT Primitive

A `THOUGHT` is the atomic unit of cognition:

> **THOUGHT = Prompt + Context + LLM + Parsing + Validation**

You define *what* the agent should think about (the prompt), and
THOUGHT handles building the messages, calling the LLM, parsing the
response, validating it, and retrying if necessary.

The universal pattern: `memory = thought(memory)`

In [17]:
print_heading("3.1  A simple THOUGHT")

# Start with a fresh memory for clarity.
mem = MEMORY()
mem.add_msg("user", "Explain event sourcing in 2 sentences.", channel="cli")

# Create a THOUGHT.  The prompt uses {last_user_msg} which THOUGHT
# automatically resolves from memory.
respond = THOUGHT(
    name="respond",
    llm=llm,
    prompt="You are a knowledgeable software architect. Answer clearly: {last_user_msg}",
)

# Execute it — this calls the LLM and stores the result.
mem = respond(mem)

# The result is stored in memory under "{name}_result".
result = mem.get_var("respond_result")
print("Response:")
print(result)


══════════════════════════════════════════════════════════════════════
  3.1  A simple THOUGHT
══════════════════════════════════════════════════════════════════════

Response:
Event sourcing is an architectural pattern that involves storing the history of an application's state as a sequence of events, allowing for the reconstruction of the current state by replaying these events in the correct order. By capturing all changes to the application's state as discrete events, event sourcing enables auditing, versioning, and debugging, as well as flexible and scalable data processing and retrieval.


In [18]:
print_heading("3.2  Prompt templating with variables")

# Prompts can reference any variable in memory via {variable_name}.
mem = MEMORY()
mem.set_var("language", "Python")
mem.set_var("concept", "decorators")
mem.add_msg("user", "Teach me about this topic.", channel="cli")

explain = THOUGHT(
    name="explain",
    llm=llm,
    prompt=(
        "Explain the concept of {concept} in {language}. "
        "Keep it to 3 sentences maximum."
    ),
    required_vars=["language", "concept"],
)

mem = explain(mem)
print("Explanation:")
print(mem.get_var("explain_result"))


══════════════════════════════════════════════════════════════════════
  3.2  Prompt templating with variables
══════════════════════════════════════════════════════════════════════

Explanation:
The concept of decorators in Python refers to a special kind of function that can modify or extend the behavior of another function. Decorators are defined with the '@' symbol followed by the name of the decorator function, and they are typically used to add additional functionality to existing functions without changing their source code. In Python, decorators are a powerful tool for implementing aspects such as logging, authentication, and caching, among others.


In [19]:
print_heading("3.3  Structured output with parsing_rules")

# parsing_rules uses valid_extract to pull structured data from the
# LLM's free-text response.  You specify the shape you want and
# ThoughtFlow extracts it.

mem = MEMORY()
mem.set_var("text", (
    "Alice is a 32-year-old software engineer from Portland. "
    "She specializes in distributed systems and has 8 years of experience."
))

extract_info = THOUGHT(
    name="extract",
    llm=llm,
    prompt=(
        "Extract structured information from this text: {text}\n\n"
        "Return a Python dict with keys: name, age, location, specialty, years_experience"
    ),
    parsing_rules={"kind": "python", "format": {
        "name": "",
        "age": 0,
        "location": "",
        "specialty": "",
        "years_experience": 0,
    }},
    max_retries=3,
)

mem = extract_info(mem)
result = mem.get_var("extract_result")
print("Extracted info:")
for key, value in result.items():
    print("  {} = {}".format(key, value))


══════════════════════════════════════════════════════════════════════
  3.3  Structured output with parsing_rules
══════════════════════════════════════════════════════════════════════

Extracted info:
  name = Alice
  age = 32
  location = Portland
  specialty = distributed systems
  years_experience = 8


In [20]:
print_heading("3.4  Validation — built-in validators")

# Validators ensure the parsed result meets your requirements.
# If validation fails, THOUGHT retries automatically.

mem = MEMORY()
mem.add_msg("user", "List 5 benefits of event sourcing.", channel="cli")

list_benefits = THOUGHT(
    name="benefits",
    llm=llm,
    prompt="List exactly 5 benefits of event sourcing as a Python list of strings: {last_user_msg}",
    parsing_rules={"kind": "python", "format": []},
    validator="list_min_len:5",
    max_retries=3,
)

mem = list_benefits(mem)
result = mem.get_var("benefits_result")
print("Benefits ({}):".format(len(result)))
for i, benefit in enumerate(result, 1):
    print("  {}. {}".format(i, benefit))


══════════════════════════════════════════════════════════════════════
  3.4  Validation — built-in validators
══════════════════════════════════════════════════════════════════════

Benefits (5):
  1. Audit trail and history
  2. Improved debugging and error handling
  3. Easier implementation of undo/redo functionality
  4. Better support for concurrency and parallel processing
  5. Enhanced data analytics and reporting capabilities


In [21]:
print_heading("3.5  Validation — custom validator function")

# You can pass a callable as the validator for full control.
# It must return (True, "") on success or (False, "reason") on failure.

def validate_haiku(result):
    """Check that the result has exactly 3 lines (a haiku)."""
    if not isinstance(result, str):
        return False, "Expected a string"
    lines = [l for l in result.strip().split("\n") if l.strip()]
    if len(lines) != 3:
        return False, "Haiku must have exactly 3 lines, got {}".format(len(lines))
    return True, ""

mem = MEMORY()
mem.set_var("subject", "machine learning")

write_haiku = THOUGHT(
    name="haiku",
    llm=llm,
    prompt=(
        "Write a haiku about {subject}. "
        "Output ONLY the three lines of the haiku, nothing else."
    ),
    validation=validate_haiku,
    max_retries=3,
)

mem = write_haiku(mem)
print("Haiku about machine learning:")
print(mem.get_var("haiku_result"))


══════════════════════════════════════════════════════════════════════
  3.5  Validation — custom validator function
══════════════════════════════════════════════════════════════════════

Haiku about machine learning:
Algorithms dance
Learning from the data sea
Intelligence born


In [22]:
print_heading("3.6  Non-LLM operations — memory_query")

# Not every THOUGHT needs an LLM.  The memory_query operation
# retrieves variables from memory without making any API call.

mem = MEMORY()
mem.set_var("user_name", "Alice")
mem.set_var("session_id", "sess_001")
mem.set_var("topic", "event sourcing")

gather_context = THOUGHT(
    name="context",
    operation="memory_query",
    required_vars=["user_name", "session_id", "topic"],
)

mem = gather_context(mem)
context = mem.get_var("context_result")
print("Gathered context (no LLM call):")
for key, value in context.items():
    print("  {} = {}".format(key, value))


══════════════════════════════════════════════════════════════════════
  3.6  Non-LLM operations — memory_query
══════════════════════════════════════════════════════════════════════

Gathered context (no LLM call):
  user_name = Alice
  session_id = sess_001
  topic = event sourcing


In [23]:
print_heading("3.7  Non-LLM operations — variable_set")

# The variable_set operation writes values into memory.
# Useful for initializing state at the start of a workflow.

mem = MEMORY()

init_session = THOUGHT(
    name="init",
    operation="variable_set",
    prompt={
        "session_active": True,
        "turn_count": 0,
        "research_phase": "discovery",
    },
)

mem = init_session(mem)
print("Variables after init:")
for key, value in mem.get_all_vars().items():
    if not key.endswith("_result"):
        print("  {} = {}".format(key, value))


══════════════════════════════════════════════════════════════════════
  3.7  Non-LLM operations — variable_set
══════════════════════════════════════════════════════════════════════

Variables after init:
  session_active = True
  turn_count = 0
  research_phase = discovery


In [24]:
print_heading("3.8  Pre/post hooks")

# Hooks let you run custom logic before or after a THOUGHT executes.

def before_thought(thought, memory, vars, **kwargs):
    """Increment the turn counter before each execution."""
    current = memory.get_var("turn_count") or 0
    memory.set_var("turn_count", current + 1)

def after_thought(thought, memory, result, error):
    """Print a summary after execution."""
    turn = memory.get_var("turn_count")
    status = "OK" if error is None else "ERROR: {}".format(error)
    print("  [hook] Turn {} complete — status: {}".format(turn, status))

mem = MEMORY()
mem.set_var("turn_count", 0)
mem.add_msg("user", "What is CQRS?", channel="cli")

hooked_thought = THOUGHT(
    name="hooked",
    llm=llm,
    prompt="Answer briefly: {last_user_msg}",
    pre_hook=before_thought,
    post_hook=after_thought,
)

mem = hooked_thought(mem)
print("\nResponse:", mem.get_var("hooked_result")[:120], "...")
print("Turn count:", mem.get_var("turn_count"))


══════════════════════════════════════════════════════════════════════
  3.8  Pre/post hooks
══════════════════════════════════════════════════════════════════════

  [hook] Turn 1 complete — status: OK

Response: CQRS (Command Query Responsibility Segregation) is a software design pattern that separates an application's responsibil ...
Turn count: 1


---
## Recap

You have now used all three foundation primitives:

| Primitive | Purpose | Key pattern |
|-----------|---------|-------------|
| **LLM** | Unified interface to language models | `response = llm.call(messages)` |
| **MEMORY** | Event-sourced state container | `memory.set_var(k, v)` / `memory.get_var(k)` |
| **THOUGHT** | Atomic unit of cognition | `memory = thought(memory)` |

Next: **Script 02** — Decisions, Plans, and Actions.

In [25]:
#[END!]   